In [30]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd 

from pydataset import data

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sns

import graphviz
from graphviz import Graph

import acquire

from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score
from sklearn.metrics import classification_report, confusion_matrix

In [31]:
telco = acquire.get_telco_data()
df = telco
df.head()

,payment_type_id,internet_service_type_id,contract_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,2,1,3,0016-QLJIS,Female,0,Yes,Yes,65,Yes,...,Yes,Yes,Yes,Yes,90.45,5957.9,No,Two year,DSL,Mailed check
1,4,1,3,0017-DINOC,Male,0,No,No,54,No,...,Yes,Yes,No,No,45.20,2460.55,No,Two year,DSL,Credit card (automatic)
2,3,1,3,0019-GFNTW,Female,0,No,No,56,No,...,Yes,No,No,No,45.05,2560.1,No,Two year,DSL,Bank transfer (automatic)
3,4,1,3,0056-EPFBG,Male,0,Yes,Yes,20,No,...,Yes,No,No,Yes,39.40,825.4,No,Two year,DSL,Credit card (automatic)
4,3,1,3,0078-XZMHT,Male,0,Yes,No,72,Yes,...,Yes,Yes,Yes,Yes,85.15,6316.2,No,Two year,DSL,Bank transfer (automatic)


In [ ]:
df.isnull().mean()
#No nulls

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.churn.value_counts()

#### Initial Focus

In [ ]:
df.contract_type.value_counts()

In [ ]:
df.internet_service_type.value_counts()

In [ ]:
df.churn.value_counts()

In [ ]:
df.churn.value_counts('internet_service_type')

In [ ]:
df.gender.groupby

In [ ]:
df.groupby('phone_service').churn.agg('count')

In [ ]:
df.groupby('senior_citizen').churn.agg('count')

In [ ]:
df.groupby('contract_type').churn.agg('count')

In [ ]:
df = df.drop_duplicates()
df

In [ ]:
df['total_charges'].value_counts()
#we have 11 values that have an empty string:

df['total_charges'] = df['total_charges'].str.strip()
df['total_charges'] = df['total_charges'].replace('', 0)
df['total_charges'].value_counts()
df['total_charges'] = df['total_charges'].astype('float64')
df['total_charges']

In [ ]:
df['senior_citizen'].value_counts()

In [ ]:
df.head()

In [ ]:
df.total_charges

## Sample questions

### Are customers with DSL more or less likely to churn?
### What month are customers most likely to churn and does that depend on their contract type?
### Is there a service that is associated with more churn than expected?
### Do customers who churn have a higher average monthly spend than those who don't?

In [ ]:
###### PRIMARY FUNCTION!!!!

In [34]:
#preliminary clean delco:
def prep_telco(df):

    df = df.drop_duplicates()  #Drop duplicates
    
    #11 values are blank strings for total charges representing 11 customers at 0 tenure
    df['total_charges'] = df['total_charges'].str.strip() #eliminate white space
    df['total_charges'] = df['total_charges'].replace('', 0) #replace empty values as 0
    df['total_charges'] = df['total_charges'].astype('float64')

    #15 columns encoded to '0' = No, '1'= Yes
    #contract type MtM = 0, 1year = 1, 2year =2
    #concated manual billing = 0 , automatic billing = 1
    df['gender'] = df.gender.replace({'Female': 0, 'Male': 1})
    df['partner'] = df.partner.replace({'Yes': 1, 'No': 0})
    df['dependents'] = df.dependents.replace({'Yes': 1, 'No': 0})
    df['phone_service'] = df.phone_service.replace({'Yes': 1, 'No': 0})
    df['churn'] = df.churn.replace({'Yes': 1, 'No': 0})
    df['multiple_lines'] = df.multiple_lines.replace({'No': 0, "Yes": 1, "No phone service": 0})
    df['online_secuirty'] = df.online_security.replace({'No': 0, "Yes": 1, "No internet service": 0})
    df['online_backup'] = df.online_backup.replace({'No': 0, "Yes": 1, "No internet service": 0})
    df['device_protection'] = df.device_protection.replace({'No': 0, "Yes": 1, "No internet service": 0})
    df['tech_support'] = df.tech_support.replace({'No': 0, "Yes": 1, "No internet service": 0})
    df['streaming_tv'] = df.streaming_tv.replace({'No': 0, "Yes": 1, "No internet service": 0})
    df['streaming_movies'] = df.streaming_movies.replace({'No': 0, "Yes": 1, "No internet service": 0})
    df['paperless_billing'] = df.paperless_billing.replace({'Yes': 1, 'No': 0})
    #df['contract_type'] = df.contract_type.replace({'Month-to-month': 0, 'One year': 1, 'Two year': 2})
    df['payment_type'] = df.payment_type.replace({'Mailed check': 0, 'Credit card (automatic)': 1, 
                                               'Bank transfer (automatic)': 1,  'Electronic check': 0})


    # creating dummy values for payment, internet, & contract (*important values*)
    dummy_df = pd.get_dummies(df[['payment_type',"internet_service_type","contract_type"]])
    df = pd.concat([df, dummy_df], axis=1)


    columns_to_rename = {'contract_type': 'contract',
                   'internet_service_type': 'internet'} # Renaming columns
    df = df.rename(columns=columns_to_rename) 
    
    return df
    
    
#drop customer_id

In [35]:
df = prep_telco(df)
df.head()

,payment_type_id,internet_service_type_id,contract_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,internet,payment_type,online_secuirty,payment_type,internet_service_type_DSL,internet_service_type_Fiber optic,internet_service_type_None,contract_type_Month-to-month,contract_type_One year,contract_type_Two year
0,2,1,3,0016-QLJIS,0,0,1,1,65,1,...,DSL,0,1,0,1,0,0,0,0,1
1,4,1,3,0017-DINOC,1,0,0,0,54,0,...,DSL,1,1,1,1,0,0,0,0,1
2,3,1,3,0019-GFNTW,0,0,0,0,56,0,...,DSL,1,1,1,1,0,0,0,0,1
3,4,1,3,0056-EPFBG,1,0,1,1,20,0,...,DSL,1,1,1,1,0,0,0,0,1
4,3,1,3,0078-XZMHT,1,0,1,0,72,1,...,DSL,1,0,1,1,0,0,0,0,1


In [21]:
dummy_df = pd.get_dummies('internet_service_type', 'contract_type_id')


df = pd.concat([df, dummy_df], axis=1)
df.head()

#    dummy_df = pd.get_dummies(df[['gender', 'contract','internet', 'payment_type']])


,payment_type_id,internet_service_type_id,contract_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type,online_secuirty,contract_type_id_internet_service_type,contract_type_id_internet_service_type
0,2,1,3,0016-QLJIS,0,0,1,1,65,1,...,1,90.45,5957.90,0,2,DSL,0,1,1.0,1.0
1,4,1,3,0017-DINOC,1,0,0,0,54,0,...,0,45.20,2460.55,0,2,DSL,1,1,NaN,NaN
2,3,1,3,0019-GFNTW,0,0,0,0,56,0,...,0,45.05,2560.10,0,2,DSL,1,1,NaN,NaN
3,4,1,3,0056-EPFBG,1,0,1,1,20,0,...,1,39.40,825.40,0,2,DSL,1,1,NaN,NaN
4,3,1,3,0078-XZMHT,1,0,1,0,72,1,...,1,85.15,6316.20,0,2,DSL,1,0,NaN,NaN


In [27]:
dummy_df = pd.get_dummies(df[['payment_type',"internet_service_type","contract_type"]])
df = pd.concat([df, dummy_df], axis=1)
df.head()

,payment_type_id,internet_service_type_id,contract_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,contract_type_One year,contract_type_Two year,payment_type,payment_type,internet_service_type_DSL,internet_service_type_Fiber optic,internet_service_type_None,contract_type_Month-to-month,contract_type_One year,contract_type_Two year
0,2,1,3,0016-QLJIS,0,0,1,1,65,1,...,0,1,0,0,1,0,0,0,0,1
1,4,1,3,0017-DINOC,1,0,0,0,54,0,...,0,1,1,1,1,0,0,0,0,1
2,3,1,3,0019-GFNTW,0,0,0,0,56,0,...,0,1,1,1,1,0,0,0,0,1
3,4,1,3,0056-EPFBG,1,0,1,1,20,0,...,0,1,1,1,1,0,0,0,0,1
4,3,1,3,0078-XZMHT,1,0,1,0,72,1,...,0,1,1,1,1,0,0,0,0,1


In [29]:
columns_to_rename = {'gender': 'is_male',
                     'payment_type_id': 'payment_id',
                     'contract_type_id': 'contract_type',
                     'internet_service_type_DSL': 'DSL',
                     'internet_service_type_Fiber optic': 'fiber_optic',
                     'internet_service_type_None': 'no_internet',
                     'contract_type_Month-to-month': 'month_to_month',
                     'contract_type_One year': 'one_year_contract',
                     'contract_type_Two year': 'two_year_contract'}
df = df.rename(columns = columns_to_rename)
df.head()

,payment_id,internet_service_type_id,contract_type,customer_id,is_male,senior_citizen,partner,dependents,tenure,phone_service,...,one_year_contract,two_year_contract,payment_type,payment_type,DSL,fiber_optic,no_internet,month_to_month,one_year_contract,two_year_contract
0,2,1,3,0016-QLJIS,0,0,1,1,65,1,...,0,1,0,0,1,0,0,0,0,1
1,4,1,3,0017-DINOC,1,0,0,0,54,0,...,0,1,1,1,1,0,0,0,0,1
2,3,1,3,0019-GFNTW,0,0,0,0,56,0,...,0,1,1,1,1,0,0,0,0,1
3,4,1,3,0056-EPFBG,1,0,1,1,20,0,...,0,1,1,1,1,0,0,0,0,1
4,3,1,3,0078-XZMHT,1,0,1,0,72,1,...,0,1,1,1,1,0,0,0,0,1


In [ ]:
columns_to_rename = {'contract_type': 'contract',
                   'internet_service_type': 'internet'} # Renaming columns
df = df.rename(columns=columns_to_rename) 

In [20]:
df['contract_type_id_internet_service_type'].sort_values()

0       1.0
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
7038    NaN
7039    NaN
7040    NaN
7041    NaN
7042    NaN
Name: contract_type_id_internet_service_type, Length: 7043, dtype: float64